In [6]:
import plotly.express as px
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

def filterCleanCPT(path, var ='FTE', municipality='CPT'):
    df = pd.read_csv(path)
    df = df[df.CAT_B == 'CPT']
    df[var] = df[var].str.replace('<10','0').astype('float')
    return df

def checkAllFTEGrowth(years=12,group='SIC7_1d',var='FTE'):
    sectors = filterCleanCPT('../cptData/Municipal_FTE_Industry5d_Monthly.csv')
    change = []
    df = sectors.groupby([group,'month'])[var].sum().reset_index()
    for sector in sectors[group].unique():
        diffDf = pd.DataFrame(df[df[group]==sector].sort_values('month')['FTE'].diff(years))
        diffDf.columns = [var+'_delta']
        diffDf['pct_change'] = pd.DataFrame(df[df[group]==sector].sort_values('month')['FTE'].pct_change(12))
        diffDf["month"]  = sorted(sectors.month.unique())
        diffDf[group] = sector
        change.append(diffDf)

    change = pd.concat(change,axis=0).set_index(['month','SIC7_1d'])
    df.set_index(['month','SIC7_1d'],inplace=True)
    change = pd.merge(df,change,left_index=True,right_index=True).reset_index()
    key  = pd.read_csv('../keys/key1d.csv')
    change = pd.merge(change,key, right_on="sic7_1d_numeric",left_on="SIC7_1d")
    return change.sort_values('month')

change = checkAllFTEGrowth(years=1,group='SIC7_1d',var='FTE')
figDf = change.pivot(index='month',columns='sic7_1d',values='FTE').sort_index().diff(12).iloc[-72:,:].stack().reset_index()
figDf.columns = ['Month','Sector','FTE change y-o-y']
fig = px.bar(figDf,x='Month',y='FTE change y-o-y',
color='Sector',
title='Year-on-Year Changes in FTE Employment',
height = 600,
color_discrete_sequence=px.colors.qualitative.Alphabet)
fig.update_layout(title_x=0.5,
                  width=1200,
                  height=600,)
fig.add_shape(
    type='rect',
     x0=figDf['Month'].min(), x1=figDf['Month'].max(),
     y0=-20000, y1=0,
    fillcolor='tomato',
    opacity=0.2,
    line=dict(color='tomato'),
    layer='below'  # Set the layer to 'below' to place the shape behind the traces
)


1. 

In [5]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
path = '../Data/Municipal_FTE_Youth_Sex_Monthly.csv'
youthDf =  filterCleanCPT(path, var ='FTE', municipality='CPT')
youthDf = youthDf[youthDf.CAT_B =='CPT']
youthDf.FTE = youthDf.FTE.astype('float')
youthDf.Youth.unique()

keep = [x  in ['[15, 25)', '[15.0, 25.0)', '[25, 35)', '[25.0, 35.0)'] for x in youthDf.Youth]
youthDf['Youth'] = keep
youthDf = youthDf.groupby(['month','Youth','Sex'])['FTE'].sum().reset_index()
youthDf['FTE change y-o-y'] = None
youthDf['FTE % change y-o-y'] = None
youthDf['Age'] = 'Over 35 Y.O.'
youthDf.Age[youthDf.Youth] =  "Under 35 Y.O."

gender = "F"
youth = True
for gender in ['F','M','Unknown']:
    for youth in [True,False]:
        selection = np.logical_and(youthDf.Sex == gender, youthDf.Youth == youth)
        youthDf['FTE change y-o-y'][selection] = youthDf[selection].sort_values('month')['FTE'].diff(12)
        youthDf['FTE % change y-o-y'][selection] = youthDf[selection].sort_values('month')['FTE'].pct_change(12)

import plotly.express as px



youthDf['Group'] =youthDf['Sex'] +" "+youthDf['Age']
colorMap = {'F Over 35 Y.O.':"Red",
 'F Under 35 Y.O.':"Pink",
 'M Over 35 Y.O.':"Blue",
 'M Under 35 Y.O.':"#ADD8E6",
 'Unknown Over 35 Y.O.':"Black",
 'Unknown Under 35 Y.O.': "#808080"}

figDf = youthDf.dropna().iloc[-12*3*3*2:]
fig = px.bar(figDf,x='month',y='FTE change y-o-y',
color='Group',
color_discrete_map=colorMap,
title='Year-on-Year Changes in FTE Employment by Gender and Age',
height = 600,
color_discrete_sequence=px.colors.qualitative.Alphabet)
fig.update_layout(title_x=0.5,
                width=1200,
                  height=600,)
fig.add_shape(
    type='rect',
     x0=figDf['month'].min(), x1=figDf['month'].max(),
     y0=-70000, y1=0,
    fillcolor='tomato',
    opacity=0.2,
    line=dict(color='tomato'),
    layer='below'  # Set the layer to 'below' to place the shape behind the traces
)
fig.show()


In [6]:

df = pd.read_csv('Data/Municipal_FTE.csv')
# df.FTE = df.FTE.str.replace("<10","0").astype('float')
df.dropna(inplace=True)
df = df[df.CAT_B.str.len() == 3]
df = df.pivot(index='TaxYear',columns='CAT_B',values='FTE')
fig = px.bar(df.iloc[-1].sort_values(ascending=False))
fig.update_layout(title='Full Time Equivalent Employment by Metro 2021/22 Tax Year',
                  title_x=0.5,
                  width=1200,
                  height=600,)
fig.update_layout(
    xaxis_title="Metro",
    yaxis_title="FTE Employment",
    showlegend=False
)
fig.show()
df

CAT_B,BUF,CPT,EKU,ETH,JHB,MAN,NMA,TSH
TaxYear,,,,,,,,
2014,132709.982658,1.177195e+06,796195.247624,828593.185431,1.980432e+06,143957.586080,216133.063434,9.496908e+05
2015,136314.045785,1.243308e+06,811807.577205,850317.808312,2.025550e+06,147757.805693,211685.253823,9.708949e+05
2016,138722.325202,1.291777e+06,789662.755717,846556.905882,2.007198e+06,148702.164472,212270.163700,1.017963e+06
2017,136626.290824,1.327925e+06,850650.394988,837911.814586,2.043079e+06,149250.789878,212129.000243,1.025536e+06
2018,140413.467010,1.351985e+06,854545.852799,929297.990829,1.995856e+06,149999.461021,210989.232947,1.034240e+06
2019,141392.400144,1.376692e+06,856605.747060,912591.498710,2.065008e+06,150651.200615,214459.160458,1.057208e+06
2020,147502.161766,1.429208e+06,870037.527271,923509.832716,2.075616e+06,156446.602745,217003.813165,1.086584e+06
2021,144131.494639,1.379051e+06,846389.457610,906357.725570,2.019031e+06,151111.285867,210808.180995,1.072507e+06
2022,141780.336367,1.373833e+06,818637.291069,911150.071400,1.963611e+06,148137.072457,208524.026847,1.065507e+06


In [7]:
import pandas as pd
import plotly.express as px
df = pd.read_csv('Data/Municipal_MedianIncome.csv')
# df.FTE = df.FTE.str.replace("<10","0").astype('float')
df.dropna(inplace=True)
df = df[df.CAT_B.str.len() == 3]
metros = {"CPT":"Cape Town",
          "ETH":"Ethekwini",
          "JHB":"Johannesburg",
          "NMA":"Nelson Mandela Bay",
            "TSH":"Tshwane",
            "BUF":"Buffalo City",
            "EKU":"Ekurhuleni",
            "MAN":"Mangaung"}
df['Metro'] = df.CAT_B.map(metros)

df = df.pivot(index='TaxYear',columns='Metro',values='FTE')

df = (df / df.iloc[0]) * 100.

fig = px.line(df)
fig.update_layout(title="FTE Employment by Metro (2014=1)",
                  title_x=0.5,
                  height=600,
                  width=1200)

fig.update_layout(
    xaxis_title="Metro",
    # yaxis_title="ZAR",
    # showlegend=False
    )

fig.update_layout(
    xaxis_title="Tax Year",
    # yaxis_title="ZAR",
    showlegend=True
)
# fig.show()
fig.show()

In [8]:
import pandas as pd
import plotly.express as px
df = pd.read_csv('Data/Municipal_MedianIncome.csv')
# df.FTE = df.FTE.str.replace("<10","0").astype('float')
df.dropna(inplace=True)
df = df[df.CAT_B.str.len() == 3]
metros = {"CPT":"Cape Town",
          "ETH":"Ethekwini",
          "JHB":"Johannesburg",
          "NMA":"Nelson Mandela Bay",
            "TSH":"Tshwane",
            "BUF":"Buffalo City",
            "EKU":"Ekurhuleni",
            "MAN":"Mangaung"}
df['Metro'] = df.CAT_B.map(metros)
df = df.pivot(index='TaxYear',columns='Metro',values='MedianIncome')
df["CPI"]=[69.3,72.0,77.,81.9,85.2,88.6,92.8,95.4,100.8]
df.CPI = df.CPI / df.CPI.iloc[0]
df = df.T   / df.T.iloc[-1] 
fig = px.line(df.iloc[:-1].T)
fig.update_layout(title="Monthly Real Median Income by Metro",
                  title_x=0.5,
                  height=500,
                  width=1200)

# To change the x and y axis labels
fig.update_xaxes(title_text='Tax Year')
fig.update_yaxes(title_text='Real Median Income (2014 Rands)')

fig.show()

In [1]:
import pandas as pd

df = pd.read_csv('../Data/Municipal_FTE_Industry5d.csv')
df.FTE = df.FTE.str.replace("<10","0").astype('float')
df = df[df.CAT_B=='CPT']
df = df.groupby(['SIC7_1d','TaxYear'])['FTE'].sum().reset_index()
df = df.pivot(index='TaxYear',columns = "SIC7_1d",values="FTE").sort_index()


df.fillna(0,inplace=True)

df = pd.concat((pd.Series(df.stack()),pd.Series(df.diff().stack())),axis=1)
df = df.reset_index()

df.columns = ['Tax Year','SIC7_1d',"FTE",'FTE Added']
# key = pd.read_excel("../keys/sic7codes_wide.xlsx")
# df = pd.merge(df, key[[ 'sic7_1d_numeric','sic7_1d_description']].drop_duplicates(), left_on='SIC7_1d',right_on='sic7_1d_numeric')

import plotly.express as px
key = pd.read_csv('../keys/key1d.csv')
df = pd.merge(df,key,right_on='sic7_1d_numeric',left_on='SIC7_1d').dropna()

df


figDf = df[df['Tax Year'] <= 2022].copy()
figDf.rename(columns={'sic7_1d':'Sector'},inplace=True)
figDf = figDf.groupby(['Tax Year','Sector'])['FTE Added'].sum().reset_index()
fig = px.bar(figDf,x = 'Tax Year',y='FTE Added',color='Sector')
fig.update_layout(title = 'Change in FTE Count by Sector',
                        title_x = 0.5,
                        width=1200,
                        height=600)


# Update the layout to shade the negative axis
fig.update_layout(
    shapes=[
        dict(
            type='rect',
            xref='paper', x0=0, x1=1,
            yref='y', y0=0, y1=-70000,
            fillcolor='red',
            opacity=0.2,
            layer='below',
            line_width=0
        )
    ]
)
fig.show()

In [10]:
df = pd.read_csv('cptData/Municipal_MedianIncome.csv')
# df.FTE = df.FTE.str.replace("<10","0").astype('float')
df.dropna(inplace=True)
df.set_index('TaxYear',inplace=True)
df = df[['FTE','MedianIncome']]

df.columns = ['FTE','Nominal Median Income']
df["CPI"]=[69.3,72.0,77.,81.9,85.2,88.6,92.8,95.4,100.8]
df['CPI'] = df['CPI'] / df['CPI'].iloc[-1]
df['Real Median Income (2022)'] = df['Nominal Median Income'] / df['CPI']
fig = px.line(df[['Nominal Median Income','Real Median Income (2022)']])
fig.update_layout(title='City of Cape Town Median Income',
                  title_x=0.5,
                  height=600,
                  width=1200)
fig.update_xaxes(title_text='Tax Year')
fig.update_yaxes(title_text='Rand (2022)')

In [11]:
import pandas as pd
import plotly.express as px
df = pd.read_csv('Data/Municipal_MedianIncome.csv')
# df.FTE = df.FTE.str.replace("<10","0").astype('float')
df.dropna(inplace=True)
df = df[df.CAT_B.str.len() == 3]
metros = {"CPT":"Cape Town",
          "ETH":"Ethekwini",
          "JHB":"Johannesburg",
          "NMA":"Nelson Mandela Bay",
            "TSH":"Tshwane",
            "BUF":"Buffalo City",
            "EKU":"Ekurhuleni",
            "MAN":"Mangaung"}
df['Metro'] = df.CAT_B.map(metros)
df = df.pivot(index='TaxYear',columns='Metro',values='MedianIncome')
# print(df)
df["CPI"]=[69.3,72.0,77.,81.9,85.2,88.6,92.8,95.4,100.8]
df.CPI = df.CPI / df.CPI.iloc[-1]
df = df.T   / df.T.iloc[-1] 
fig = px.line(df.iloc[:-1].T)
fig.update_layout(title="Monthly Real Median Income by Metro (2022)",
                  title_x=0.5,
                  height=600,
                  width=1200)

# To change the x and y axis labels
fig.update_xaxes(title_text='Tax Year')
fig.update_yaxes(title_text='Rand (2022)')

fig.show()


In [3]:
df = pd.read_csv('Data/Municipal_FTE.csv')
# df.FTE = df.FTE.str.replace("<10","0").astype('float')
df.dropna(inplace=True)
df = df[df.CAT_B.str.len() == 3]
metros = {"CPT":"Cape Town",
          "ETH":"Ethekwini",
          "JHB":"Johannesburg",
          "NMA":"Nelson Mandela Bay",
            "TSH":"Tshwane",
            "BUF":"Buffalo City",
            "EKU":"Ekurhuleni",
            "MAN":"Mangaung"}
df['Metro'] = df.CAT_B.map(metros)
df = df.pivot(index='TaxYear',columns='Metro',values='FTE')
fig = px.bar(df.iloc[-1].sort_values(ascending=False))
fig.update_layout(title='Full Time Equivalent Employment by Metro 2021/22 Tax Year',
                  title_x=0.5,
                  width=1200,
                  height=600,)
fig.update_layout(
    xaxis_title="Metro",
    yaxis_title="FTE Employment",
    showlegend=False
)
fig.show()

# Maps

In [ ]:
import os
import pandas as pd
import numpy as np
import geopandas as gpd
import plotly.express as px
# os.chdir('/data/workspace_files/maps')
import geopandas as gpd
import numpy as np
gpd_df= gpd.read_file('../Shapefiles/Cape Town.shp')
hex_df = pd.read_csv("../Data/hex7_FTE.csv")
hex_df.FTE = hex_df.FTE.replace("<10",np.nan).astype("float")
gpd_df = gpd_df.merge(hex_df,on="hex7")

# gpd_df[gpd_df.TaxYear==2022].FTE - gpd_df[gpd_df.TaxYear==2014].FTE
out = gpd_df.pivot(index="TaxYear",columns="hex7",values="FTE")
gpd_df = pd.merge(gpd_df[gpd_df.TaxYear==2022],pd.Series((out.iloc[1]-out.iloc[0]),name='FTE Growth'),left_on="hex7",right_index=True)
gpd_df

x = gpd_df.geometry.centroid.x.median()
y = gpd_df.geometry.centroid.y.median() 

# print(gpd_df)

gpd_df["FTE Abs Growth"] = gpd_df['FTE Growth'].abs()
gpd_df = gpd_df.sort_values("FTE Growth", ascending=False).head(100)

# gpd_df['percentile'] = pct_map(gpd_df, groupVar = 'taxyear', rankVar = "FTE")
# print(gpd_df)

# gpd_df.set_index('hex7',inplace=True)
fig = px.choropleth_mapbox(gpd_df,
                   geojson=gpd_df.geometry,
                   locations=gpd_df.index,
                   color="FTE Growth",
                    mapbox_style = "open-street-map",
                    # animation_frame='taxyear',
                    # color_continuous_midpoint=0,
                    # color_continuous_scale="RdBu",
                    color_continuous_scale="dense",
                    center = {"lat":y, "lon":x},
                    title="Top Areas for FTE Employment Creation 2014-2022",
                   opacity=0.5, 
width=1200,height=600,zoom=9)
fig.update_layout(
    # margin={"r":0,"t":30,"l":0,"b":30},
                   title_x=0.5
)
# fig.update_geos(fitbounds="locations")
fig.show()

In [ ]:
import os
import pandas as pd
import numpy as np
import geopandas as gpd
import plotly.express as px
# os.chdir('/data/workspace_files/maps')
import geopandas as gpd
import numpy as np
gpd_df= gpd.read_file('../Shapefiles/Cape Town.shp')
hex_df = pd.read_csv("../Data/hex7_Establishments.csv")
hex_df.Establishments = hex_df.Establishments.replace("<10",np.nan).astype("float")
gpd_df = gpd_df.merge(hex_df,on="hex7")



# gpd_df[gpd_df.TaxYear==2022].FTE - gpd_df[gpd_df.TaxYear==2014].FTE
out = gpd_df.pivot(index="TaxYear",columns="hex7",values="Establishments")
gpd_df = pd.merge(gpd_df[gpd_df.TaxYear==2022],pd.Series((out.iloc[1]-out.iloc[0]),name='Establishment Growth'),left_on="hex7",right_index=True)
gpd_df

x = gpd_df.geometry.centroid.x.median()
y = gpd_df.geometry.centroid.y.median() - 0.17

# print(gpd_df)

gpd_df["Establishment Abs Growth"] = gpd_df['Establishment Growth'].abs()
gpd_df = gpd_df.sort_values("Establishment Growth", ascending=False).head(100)

# gpd_df['percentile'] = pct_map(gpd_df, groupVar = 'taxyear', rankVar = "FTE")
# print(gpd_df)

# gpd_df.set_index('hex7',inplace=True)
fig = px.choropleth_mapbox(gpd_df,
                   geojson=gpd_df.geometry,
                   locations=gpd_df.index,
                   color='Establishment Growth',
                    mapbox_style = "open-street-map",
                    # animation_frame='taxyear',
                    # color_continuous_midpoint=0,
                    # color_continuous_scale="RdBu",
                    color_continuous_scale="dense",
                    center = {"lat":y, "lon":x},
                    title="Top Areas for Establishment Creation 2014-2022",
                   opacity=0.5, 
width=1200,height=600,zoom=9)
fig.update_layout(
    # margin={"r":0,"t":30,"l":0,"b":30},
                   title_x=0.5
)
# fig.update_geos(fitbounds="locations")
fig.show()